# Treinamento Focado (Simulador Macro Puro)

**Estratégia Final de Modelagem:**
Após os testes comparativos, optou-se por uma abordagem de **Features Selecionadas (White-box)**.
Removemos deliberadamente as variáveis de histórico de inadimplência (`lags` do target) para impedir que o modelo se apoie no passado recente.

**Objetivo:**
Criar modelos que respondam **exclusivamente** às variações macroeconômicas e sazonais.

**Algoritmo Escolhido:** `Ridge Regression` (Linear Regularizado).

In [1]:
# Bibliotecas Importadas
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Configuração Visual
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Caminhos
base_path = "../data/processed/"
model_path = "../models/"
os.makedirs(model_path, exist_ok=True)

# Carregar Dados (Versão Final V3)
if os.path.exists(f"{base_path}df_modelagem_v3.csv"):
    df = pd.read_csv(f"{base_path}df_modelagem_v3.csv", index_col="data", parse_dates=True)
    print(f" Dados carregados: {df.shape}")
else:
    raise FileNotFoundError("Arquivo df_modelagem_v3.csv não encontrado.")

 Dados carregados: (120, 50)


## Definição das Features

In [2]:
# Removemos qualquer ruído e qualquer "cola" do passado (inadimplência anterior).
features_focadas = [
    'selic_lag_6',      # Alavanca Principal (Juros demoram ~6 meses para bater na ponta)
    'ipca_lag_6',       # Contexto inflacionário
    'dolar_ptax_lag_6', # Câmbio (Impacto forte no Rural/Indústria)
    'mes',              # Sazonalidade (Final de ano, etc)
    'periodo_safra'     # Sazonalidade Específica do Rural
]

# Targets (Alvos)
targets = {
    "PF": "inad_pf_tot",
    "PJ": "inad_pj_tot",
    "Rural_PF": "inad_rd_pf_cr_rur_tot", 
    "Rural_PJ": "inad_rd_pj_cr_rur_tot"   
}

# Verificação de segurança
missing_cols = [c for c in features_focadas if c not in df.columns]
if missing_cols:
    print(f" ATENÇÃO: As seguintes colunas não existem no DF e serão ignoradas: {missing_cols}")
else:
    print(f" Features definidas: {features_focadas}")

 Features definidas: ['selic_lag_6', 'ipca_lag_6', 'dolar_ptax_lag_6', 'mes', 'periodo_safra']


## Treinamento e Diagnóstico de Sensibilidade

In [3]:
resultados_analise = []

print("Iniciando calibração dos modelos...")

for nome_modelo, target_col in targets.items():
    if target_col not in df.columns: continue
    
    print(f"\n--- Calibrando Segmento: {nome_modelo} ---")
    
    # 1. Preparar Dados
    cols_to_use = [c for c in features_focadas if c in df.columns]
    
    df_temp = df.copy()
    df_temp = df_temp[cols_to_use + [target_col]].dropna()
    
    X = df_temp[cols_to_use]
    y = df_temp[target_col]
    
    # 2. Padronização
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 3. Treinamento (Ridge Regression)
    # alpha=0.5 -> Baixa regularização, deixa o modelo se ajustar bem aos dados macro
    model = Ridge(alpha=0.5) 
    model.fit(X_scaled, y)
    
    # 4. Avaliação Rápida
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    
    # 5. Diagnóstico de Impacto (Coeficientes)
    coefs = pd.DataFrame({'feature': X.columns, 'coef': model.coef_})
    
    # Tentamos pegar o peso da Selic
    try:
        peso_selic = coefs.loc[coefs['feature'].str.contains('selic'), 'coef'].values[0]
        status_selic = "Forte" if abs(peso_selic) > 0.05 else "Fraco"
    except:
        peso_selic = 0
        status_selic = "Indefinido"
        
    print(f"   > R² (Aderência Macro): {r2:.2f}")
    print(f"   > RMSE (Erro Médio): {rmse:.2f}%")
    print(f"   > Sensibilidade Selic: {peso_selic:.4f} ({status_selic})")
    
    if peso_selic < 0 and "Rural" not in nome_modelo:
        print("  AVISO: Selic negativa? Isso indicaria que juros altos baixam a inadimplência (Contraintuitivo para PF/PJ).")

    # 6. Salvamento
    # Nomes padronizados para o App encontrar fácil
    joblib.dump(model, f"{model_path}model_{nome_modelo}.pkl")
    joblib.dump(scaler, f"{model_path}scaler_{nome_modelo}.pkl")
    
    # Metadados essenciais
    pd.DataFrame(columns=X.columns).to_csv(f"{model_path}columns_{nome_modelo}.csv", index=False)
    X.iloc[-1].to_csv(f"{model_path}last_values_{nome_modelo}.csv")
    
    # Guardar para plotar depois
    resultados_analise.append((nome_modelo, y, y_pred))

print("\n Todos os modelos foram salvos e estão prontos para o App.")

Iniciando calibração dos modelos...

--- Calibrando Segmento: PF ---
   > R² (Aderência Macro): 0.81
   > RMSE (Erro Médio): 0.16%
   > Sensibilidade Selic: 0.3084 (Forte)

--- Calibrando Segmento: PJ ---
   > R² (Aderência Macro): 0.87
   > RMSE (Erro Médio): 0.24%
   > Sensibilidade Selic: 0.3819 (Forte)

--- Calibrando Segmento: Rural_PF ---
   > R² (Aderência Macro): 0.47
   > RMSE (Erro Médio): 0.53%
   > Sensibilidade Selic: -0.1000 (Forte)

--- Calibrando Segmento: Rural_PJ ---
   > R² (Aderência Macro): 0.31
   > RMSE (Erro Médio): 1.51%
   > Sensibilidade Selic: -0.8430 (Forte)

 Todos os modelos foram salvos e estão prontos para o App.
